In [10]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from albumentations import Compose, HorizontalFlip, VerticalFlip, RandomRotate90, ShiftScaleRotate, Blur, RandomBrightnessContrast

data_dir = r'C:\Users\haris\ArtificialIntelligence\MachineLearning\Projects\Angry Sad Happy'

# Define the augmentation transformations
augmentation_transform = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    RandomRotate90(p=0.5),
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
    Blur(p=0.2),
    RandomBrightnessContrast(p=0.3),
])
# Function to load and augment images in batches
def image_data_generator(batch_size=32):
    while True:
        batch_images = []
        batch_labels = []
        for emotion in ['Happy', 'Sad', 'Angry']:
            folder_path = os.path.join(data_dir, emotion)
            for filename in os.listdir(folder_path):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))  # Resize images to VGG input size
                for _ in range(batch_size):
                    augmented = augmentation_transform(image=image)
                    augmented_image = augmented['image']
                    batch_images.append(augmented_image)
                    batch_labels.append(emotion)
                    if len(batch_images) == batch_size:
                        yield np.array(batch_images), np.array(batch_labels)
                        batch_images = []
                        batch_labels = []

# Create the data generator
batch_size = 32
data_generator = image_data_generator(batch_size=batch_size)

# Create a list to store data
images = []
labels = []

# Load and augment data using the generator
num_samples_per_class = 100  # Increase the number of samples per class
for _ in range(num_samples_per_class):
    X_batch, y_batch = next(data_generator)
    images.extend(X_batch)
    labels.extend(y_batch)

# Create a list to store training and validation data separately
train_images = []
train_labels = []
val_images = []
val_labels = []

# Load and augment data using the generator and separate it into training and validation sets
num_samples_per_class = 100
num_train_samples = int(num_samples_per_class * 0.8)  # 80% for training, 20% for validation

for _ in range(num_train_samples):
    X_batch, y_batch = next(data_generator)
    train_images.extend(X_batch)
    train_labels.extend(y_batch)

for _ in range(num_samples_per_class - num_train_samples):
    X_batch, y_batch = next(data_generator)
    val_images.extend(X_batch)
    val_labels.extend(y_batch)

# Convert the list of images and labels to numpy arrays
X_train = np.array(train_images)
y_train_labels = np.array(train_labels)
X_val = np.array(val_images)
y_val_labels = np.array(val_labels)

# Encode labels into numeric form
label_to_numeric = {'Happy': 0, 'Sad': 1, 'Angry': 2}
y_train_numeric = np.array([label_to_numeric[label] for label in y_train_labels])
y_val_numeric = np.array([label_to_numeric[label] for label in y_val_labels])

# Convert labels to one-hot encoded format
num_classes = 3
y_train_one_hot = to_categorical(y_train_numeric, num_classes=num_classes)
y_val_one_hot = to_categorical(y_val_numeric, num_classes=num_classes)

# Load the pre-trained VGG16 model (without the top classification layers)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained VGG16 model except the last five layers
for layer in vgg16.layers[:-5]:
    layer.trainable = False

# Create a new model by adding custom classification layers on top of the pre-trained VGG16
model = Sequential([
    vgg16,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

Epoch 1/10
80/80 [==============================] - 18s 178ms/step - loss: 2.8514 - accuracy: 0.9645 - val_loss: 4.1043 - val_accuracy: 0.0000e+00
Epoch 2/10
80/80 [==============================] - 14s 175ms/step - loss: 0.2203 - accuracy: 0.9707 - val_loss: 57.3388 - val_accuracy: 0.0000e+00
Epoch 3/10
80/80 [==============================] - 14s 176ms/step - loss: 5.8500 - accuracy: 0.9746 - val_loss: 330.7182 - val_accuracy: 0.0000e+00
Epoch 4/10
80/80 [==============================] - 14s 177ms/step - loss: 0.6206 - accuracy: 0.9746 - val_loss: 180.5731 - val_accuracy: 0.0000e+00
Epoch 5/10
80/80 [==============================] - 14s 177ms/step - loss: 0.3323 - accuracy: 0.9734 - val_loss: 15.3327 - val_accuracy: 0.0000e+00
Epoch 6/10
80/80 [==============================] - 14s 178ms/step - loss: 1.2165 - accuracy: 0.9727 - val_loss: 1.8477 - val_accuracy: 0.0000e+00
Epoch 7/10
80/80 [==============================] - 14s 178ms/step - loss: 0.1987 - accuracy: 0.9750 - val_loss:

In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Load an image for prediction
image_path = r'C:\Users\haris\ArtificialIntelligence\MachineLearning\Projects\Angry Sad Happy\5cd88fd321000035007f6cd2.jpeg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))  # Resize to VGG input size

# Preprocess the image (use VGG16 preprocess_input)
image = preprocess_input(image)

# Expand the dimensions to match the input shape of the model
image = np.expand_dims(image, axis=0)

# Make the prediction
predicted_probs = model.predict(image)

# Get the predicted class (index with highest probability)
predicted_class = np.argmax(predicted_probs)

# Convert the numeric class to emotion label
numeric_to_label = {0: 'Angry', 1: 'Happy', 2: 'Sad'}  # Update the mapping
predicted_emotion = numeric_to_label[predicted_class]

print(f"Predicted emotion: {predicted_emotion}")
print(f"Predicted probabilities: {predicted_probs[0]}")

1/1 [==============================] - 0s 21ms/step
Predicted emotion: Happy
Predicted probabilities: [0. 1. 0.]
